1. Configuración

In [1]:
import os
import random
import numpy as np
import torch
from pathlib import Path

OUT_DIR_02 = Path("./outputs_02_mert_finetune_egfxset")

In [2]:
SEED = 42

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)

seed_everything(SEED)

In [3]:
if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print("device:", device, "| torch:", torch.__version__, "| seed:", SEED)

device: mps | torch: 2.9.1 | seed: 42


2. Carga del .csv del dataset

In [4]:
import pandas as pd
from pathlib import Path

INDEX_CSV_PATH = Path("./outputs_01_mert_base_egfxset/egfxset_index.csv")

assert INDEX_CSV_PATH.exists(), f"No existe el índice: {INDEX_CSV_PATH}"

df = pd.read_csv(INDEX_CSV_PATH)

print("Dataset cargado")
print("Número de muestras:", len(df))
print("Columnas:", list(df.columns))
print()
print(df.head())

Dataset cargado
Número de muestras: 8947
Columnas: ['path', 'tone', 'pickup', 'tone_pickup', 'string', 'fret', 'midi_pitch']

                                                path         tone  pickup  \
0  /Users/dtenreiro/Documents/TFM/EGFxSet/BluesDr...  BluesDriver  Bridge   
1  /Users/dtenreiro/Documents/TFM/EGFxSet/BluesDr...  BluesDriver  Bridge   
2  /Users/dtenreiro/Documents/TFM/EGFxSet/BluesDr...  BluesDriver  Bridge   
3  /Users/dtenreiro/Documents/TFM/EGFxSet/BluesDr...  BluesDriver  Bridge   
4  /Users/dtenreiro/Documents/TFM/EGFxSet/BluesDr...  BluesDriver  Bridge   

           tone_pickup  string  fret  midi_pitch  
0  BluesDriver__Bridge       1     0          64  
1  BluesDriver__Bridge       1     1          65  
2  BluesDriver__Bridge       1    10          74  
3  BluesDriver__Bridge       1    11          75  
4  BluesDriver__Bridge       1    12          76  


3. Split 80/20

In [5]:
from sklearn.model_selection import train_test_split

LABEL_COL = "tone_pickup"
assert LABEL_COL in df.columns, f"No existe la columna de etiqueta '{LABEL_COL}'. Columnas: {list(df.columns)}"

train_df, val_df = train_test_split(
    df,
    test_size=0.20,
    random_state=SEED,
    stratify=df[LABEL_COL]
)

print("Split OK")
print("Train:", len(train_df), "| Val:", len(val_df))
print("Clases train:", train_df[LABEL_COL].nunique(), "| Clases val:", val_df[LABEL_COL].nunique())
print("\nDistribución (train):")
print(train_df[LABEL_COL].value_counts().head(10))
print("\nDistribución (val):")
print(val_df[LABEL_COL].value_counts().head(10))

Split OK
Train: 7157 | Val: 1790
Clases train: 65 | Clases val: 65

Distribución (train):
tone_pickup
Chorus__Bridge               111
TapeEcho__Neck               111
BluesDriver__Neck            111
Sweep Echo__Middle-Neck      111
Clean__Middle-Neck           111
Sweep Echo__Bridge           111
Digital Delay__Neck          111
TubeScreamer__Middle         111
Sweep Echo__Bridge-Middle    111
Plate Reverb__Bridge         111
Name: count, dtype: int64

Distribución (val):
tone_pickup
TapeEcho__Middle-Neck        28
Spring Reverb__Neck          28
Sweep Echo__Middle           28
RAT__Middle-Neck             28
TubeScreamer__Middle-Neck    28
Flanger__Neck                28
TapeEcho__Bridge             28
Hall Reverb__Neck            28
RAT__Neck                    28
TubeScreamer__Neck           28
Name: count, dtype: int64


4. Label encoding

In [6]:
from sklearn.preprocessing import LabelEncoder

LABEL_COL = "tone_pickup"

le = LabelEncoder()
le.fit(df[LABEL_COL])

train_df = train_df.copy()
val_df   = val_df.copy()

train_df["label_id"] = le.transform(train_df[LABEL_COL])
val_df["label_id"]   = le.transform(val_df[LABEL_COL])

num_classes = len(le.classes_)

print("Label encoding OK")
print("Num clases:", num_classes)
print("Ejemplo mapping:")
for i, c in enumerate(le.classes_[:10]):
    print(f"{i} -> {c}")

Label encoding OK
Num clases: 65
Ejemplo mapping:
0 -> BluesDriver__Bridge
1 -> BluesDriver__Bridge-Middle
2 -> BluesDriver__Middle
3 -> BluesDriver__Middle-Neck
4 -> BluesDriver__Neck
5 -> Chorus__Bridge
6 -> Chorus__Bridge-Middle
7 -> Chorus__Middle
8 -> Chorus__Middle-Neck
9 -> Chorus__Neck


5. MERT

In [7]:
import torch
import torchaudio
import soundfile as sf
import torch.nn.functional as F
from pathlib import Path

from transformers import AutoModel, AutoProcessor

# Modelo MERT baseline
MERT_MODEL_NAME = "m-a-p/MERT-v1-330M"
print("Loading MERT model:", MERT_MODEL_NAME)

mert_model = AutoModel.from_pretrained(
    MERT_MODEL_NAME,
    trust_remote_code=True
).to(device)

# Processor
processor = AutoProcessor.from_pretrained(
    MERT_MODEL_NAME,
    trust_remote_code=True
)

# hidden_states
mert_model.config.output_hidden_states = True

# Config audio
MAX_SECONDS = 5.0
TARGET_SR = 24000

def load_audio(path: str | Path):
    """Carga wav -> mono -> resample a 24k -> recorta a 5s."""
    wav, sr = sf.read(str(path))
    if wav.ndim == 2:
        wav = wav.mean(axis=1)
    wav = torch.from_numpy(wav).float()
    if sr != TARGET_SR:
        wav = torchaudio.functional.resample(wav, sr, TARGET_SR)
        sr = TARGET_SR
    wav = wav[: int(sr * MAX_SECONDS)]
    return wav, sr

print("MERT loaded on device:", device)

/opt/anaconda3/envs/mert310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading MERT model: m-a-p/MERT-v1-330M


Some weights of the model checkpoint at m-a-p/MERT-v1-330M were not used when initializing MERTModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing MERTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MERTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MERTModel were not initialized from the model checkpoint at m-a-p/MERT-v1-330M and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


MERT loaded on device: mps


6. Data Set y Loaders

In [8]:
import numpy as np
import torch
import torchaudio
import soundfile as sf
from pathlib import Path

MAX_SECONDS = 5.0
TARGET_SR = 24000
TARGET_LEN = int(TARGET_SR * MAX_SECONDS)

def load_audio(path: str | Path):
    wav, sr = sf.read(str(path))

    if isinstance(wav, np.ndarray) and wav.ndim == 2:
        wav = wav.mean(axis=1)

    wav = torch.tensor(wav, dtype=torch.float32)

    if sr != TARGET_SR:
        wav = torchaudio.functional.resample(wav, sr, TARGET_SR)
        sr = TARGET_SR

    wav = torch.nan_to_num(wav, nan=0.0, posinf=0.0, neginf=0.0)

    if wav.numel() < TARGET_LEN:
        pad = TARGET_LEN - wav.numel()
        wav = torch.cat([wav, torch.zeros(pad, dtype=wav.dtype)], dim=0)
    else:
        wav = wav[:TARGET_LEN]

    return wav, sr

7. input_values está saliendo con shape (1, 8, 120000).

In [9]:
BATCH_SIZE = 8
LR = 2e-5
N_LAST_LAYERS = 4
MAX_SECONDS = 5.0

In [10]:
def collate_fn(batch):
    wavs, srs, ys = zip(*batch)

    wavs_np = [w.cpu().numpy().astype(np.float32) for w in wavs]

    inputs = processor(
        wavs_np,
        sampling_rate=srs[0],
        return_tensors="pt",
        padding=False,
        return_attention_mask=True
    )

    labels = torch.tensor(ys, dtype=torch.long)
    return inputs, labels

In [11]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

PATH_COL = "path"
LABEL_ID_COL = "label_id"

assert PATH_COL in train_df.columns, f"Falta '{PATH_COL}' en train_df. Columnas: {list(train_df.columns)}"
assert LABEL_ID_COL in train_df.columns, f"Falta '{LABEL_ID_COL}' en train_df. Columnas: {list(train_df.columns)}"

class EGFxSetDataset(Dataset):
    def __init__(self, df):
        self.paths = df[PATH_COL].astype(str).tolist()
        self.labels = df[LABEL_ID_COL].astype(int).tolist()

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        wav, sr = load_audio(self.paths[idx])
        y = self.labels[idx]
        return wav, sr, y

train_loader = DataLoader(
    EGFxSetDataset(train_df),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    collate_fn=collate_fn
)

val_loader = DataLoader(
    EGFxSetDataset(val_df),
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    collate_fn=collate_fn
)

inputs, labels = next(iter(train_loader))
print("input_values:", tuple(inputs["input_values"].shape), inputs["input_values"].dtype)
print("attention_mask:", tuple(inputs["attention_mask"].shape), inputs["attention_mask"].dtype)
print("labels:", tuple(labels.shape), labels.dtype)

input_values: (8, 120000) torch.float32
attention_mask: (8, 120000) torch.int32
labels: (8,) torch.int64


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MERTForTonePickup(nn.Module):
    def __init__(self, mert_model, num_classes: int, proj_dim: int = 256, dropout: float = 0.1):
        super().__init__()
        self.mert = mert_model

        hidden_size = getattr(self.mert.config, "hidden_size", None)
        if hidden_size is None:
            raise ValueError("No se pudo leer hidden_size de mert_model.config.hidden_size")

        self.dropout = nn.Dropout(dropout)
        self.proj = nn.Linear(hidden_size, proj_dim)
        self.classifier = nn.Linear(proj_dim, num_classes)

    def forward(self, inputs, labels=None):
        out = self.mert(**inputs)
        h = out.last_hidden_state  # (B, T, H)

        # MAX pooling temporal
        emb = h.max(dim=1).values  # (B, H)
        emb = self.dropout(emb)

        emb = F.relu(self.proj(emb))  # (B, proj_dim)
        emb = self.dropout(emb)

        logits = self.classifier(emb)
        loss = F.cross_entropy(logits, labels) if labels is not None else None
        return {"loss": loss, "logits": logits, "emb": emb}

ft_model = MERTForTonePickup(mert_model, num_classes=num_classes, proj_dim=256, dropout=0.1).to(device)

print("Wrapper OK: last layer + MAX pooling")

Wrapper OK: last layer + MAX pooling


In [13]:
N_LAST_LAYERS = 4
LR = 2e-5
WEIGHT_DECAY = 0.01

for p in ft_model.parameters():
    p.requires_grad = False

for p in ft_model.proj.parameters():
    p.requires_grad = True
for p in ft_model.classifier.parameters():
    p.requires_grad = True

mert_core = ft_model.mert

layer_stack = None
candidates = [
    ("encoder.layers", lambda m: m.encoder.layers),
    ("model.encoder.layers", lambda m: m.model.encoder.layers),
    ("transformer.layers", lambda m: m.transformer.layers),
    ("model.layers", lambda m: m.model.layers),
    ("layers", lambda m: m.layers),
]

for name, getter in candidates:
    try:
        ls = getter(mert_core)
        if hasattr(ls, "__len__"):
            layer_stack = ls
            stack_name = name
            break
    except Exception:
        pass

assert layer_stack is not None, (
    "No pude encontrar el stack de capas del encoder en este MERT"
)

num_layers = len(layer_stack)
n = min(N_LAST_LAYERS, num_layers)

for layer in layer_stack[num_layers - n:]:
    for p in layer.parameters():
        p.requires_grad = True

optimizer = torch.optim.AdamW(
    [p for p in ft_model.parameters() if p.requires_grad],
    lr=LR,
    weight_decay=WEIGHT_DECAY
)

trainable = sum(p.numel() for p in ft_model.parameters() if p.requires_grad)
total = sum(p.numel() for p in ft_model.parameters())
print(f"Unfroze last {n}/{num_layers} layers via '{stack_name}' + head/proj")
print(f"Trainable params: {trainable:,} / {total:,} ({trainable/total:.2%})")

Unfroze last 4/24 layers via 'encoder.layers' + head/proj
Trainable params: 50,664,001 / 315,708,097 (16.05%)


In [14]:
from tqdm import tqdm
import torch
from pathlib import Path
import math

EPOCHS_MAX = 30
PATIENCE = 3
GRAD_CLIP = 1.0

OUT_DIR_02 = Path("./outputs_02_mert_finetune_egfxset")
OUT_DIR_02.mkdir(parents=True, exist_ok=True)

BEST_CKPT = OUT_DIR_02 / "best.pt"
LAST_CKPT = OUT_DIR_02 / "last.pt"

def run_epoch(loader, train: bool):
    ft_model.train(train)
    total_loss, correct, total = 0.0, 0, 0

    for inputs, labels in tqdm(loader, disable=False):
        inputs = {k: v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)

        out = ft_model(inputs, labels=labels)
        loss = out["loss"]

        if train:
            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(ft_model.parameters(), GRAD_CLIP)
            optimizer.step()

        total_loss += float(loss.item()) * labels.size(0)
        preds = out["logits"].argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return total_loss / total, correct / total

history_ft = globals().get("history_ft", [])

best_val_loss = math.inf
pat_left = PATIENCE
start_epoch = 1

if LAST_CKPT.exists():
    ckpt_last = torch.load(LAST_CKPT, map_location="cpu")
    if "ft_model_state_dict" in ckpt_last:
        ft_model.load_state_dict(ckpt_last["ft_model_state_dict"], strict=True)
    if "optimizer_state_dict" in ckpt_last:
        try:
            optimizer.load_state_dict(ckpt_last["optimizer_state_dict"])
            print(f"Resumed optimizer state from {LAST_CKPT}")
        except Exception as e:
            print(f"WARNING: could not load optimizer state: {e}")
    if "history_ft" in ckpt_last:
        history_ft = ckpt_last["history_ft"]
    if "epoch" in ckpt_last:
        start_epoch = int(ckpt_last["epoch"]) + 1
    if "best_val_loss" in ckpt_last:
        best_val_loss = float(ckpt_last["best_val_loss"])
    if "pat_left" in ckpt_last:
        pat_left = int(ckpt_last["pat_left"])

    print(f"Resumed model from {LAST_CKPT} | next epoch: {start_epoch} | best_val_loss: {best_val_loss:.4f} | pat_left: {pat_left}")
else:

    if BEST_CKPT.exists():
        ckpt_best = torch.load(BEST_CKPT, map_location="cpu")
        if "best_val_loss" in ckpt_best:
            best_val_loss = float(ckpt_best["best_val_loss"])
        print(f"Loaded best_val_loss={best_val_loss:.4f} from {BEST_CKPT}")

for epoch in range(start_epoch, EPOCHS_MAX + 1):
    tr_loss, tr_acc = run_epoch(train_loader, train=True)
    va_loss, va_acc = run_epoch(val_loader, train=False)
    history_ft.append((epoch, tr_loss, tr_acc, va_loss, va_acc))

    print(f"epoch {epoch:02d} | train loss {tr_loss:.4f} acc {tr_acc:.3f} | val loss {va_loss:.4f} acc {va_acc:.3f}")

    torch.save(
        {
            "ft_model_state_dict": ft_model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "epoch": epoch,
            "history_ft": history_ft,
            "best_val_loss": best_val_loss,
            "pat_left": pat_left,
        },
        LAST_CKPT
    )

    if va_loss < best_val_loss - 1e-4:
        best_val_loss = va_loss
        pat_left = PATIENCE
        torch.save(
            {
                "ft_model_state_dict": ft_model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "epoch": epoch,
                "best_val_loss": best_val_loss,
                "history_ft": history_ft,
                "pat_left": pat_left,
                "label_classes": le.classes_.tolist(),
                "n_last_layers": int(N_LAST_LAYERS),
                "mert_model_name": MERT_MODEL_NAME,
                "proj_dim": 256,
                "pooling": "max_last_hidden_state",
            },
            BEST_CKPT
        )
        print(f"new best val_loss={best_val_loss:.4f} -> saved {BEST_CKPT}")
    else:
        pat_left -= 1
        print(f"no improvement | patience left {pat_left}/{PATIENCE}")
        if pat_left == 0:
            print("Early stopping")
            break

print("Training done.")
print("best_val_loss:", best_val_loss)
print("Best ckpt:", BEST_CKPT)
print("Last ckpt:", LAST_CKPT)

100%|█████████████████████████████████████████| 224/224 [01:48<00:00,  2.07it/s]


epoch 01 | train loss 4.0357 acc 0.038 | val loss 3.7734 acc 0.081
new best val_loss=3.7734 -> saved outputs_02_mert_finetune_egfxset/best.pt


100%|█████████████████████████████████████████| 224/224 [01:51<00:00,  2.02it/s]


epoch 02 | train loss 3.3683 acc 0.137 | val loss 2.9204 acc 0.265
new best val_loss=2.9204 -> saved outputs_02_mert_finetune_egfxset/best.pt


100%|█████████████████████████████████████████| 224/224 [01:46<00:00,  2.11it/s]


...

epoch 18 | train loss 0.0271 acc 0.992 | val loss 0.3822 acc 0.903
no improvement | patience left 2/3


100%|█████████████████████████████████████████| 224/224 [01:35<00:00,  2.35it/s]


epoch 19 | train loss 0.0208 acc 0.994 | val loss 0.5762 acc 0.864
no improvement | patience left 1/3


100%|█████████████████████████████████████████| 224/224 [01:35<00:00,  2.35it/s]


epoch 20 | train loss 0.0198 acc 0.995 | val loss 0.3704 acc 0.908
no improvement | patience left 0/3
Early stopping
Training done.
best_val_loss: 0.27758973892830185
Best ckpt: outputs_02_mert_finetune_egfxset/best.pt
Last ckpt: outputs_02_mert_finetune_egfxset/last.pt


10. Se guardan los pesos y metadata

In [15]:
import json
import numpy as np
import torch
from pathlib import Path

OUT_DIR_02 = Path("./outputs_02_mert_finetune_egfxset")
OUT_DIR_02.mkdir(parents=True, exist_ok=True)

ckpt_path = OUT_DIR_02 / "mert_ft_last4layers_headproj.pt"
meta_path = OUT_DIR_02 / "metadata.json"

torch.save(
    {
        "ft_model_state_dict": ft_model.state_dict(),
        "label_classes": le.classes_.tolist(),
        "label_col": LABEL_COL,
        "path_col": PATH_COL,
        "num_classes": int(num_classes),
        "seed": int(SEED),
        "n_last_layers": int(N_LAST_LAYERS),
    },
    ckpt_path
)

meta = {
    "mert_model_name": MERT_MODEL_NAME,
    "target_sr": TARGET_SR,
    "max_seconds": MAX_SECONDS,
    "batch_size": BATCH_SIZE,
    "lr": LR,
    "weight_decay": WEIGHT_DECAY,
    "n_last_layers": N_LAST_LAYERS,
}
if "history_ft" in globals():
    meta["history_ft"] = [
        {"epoch": int(e), "train_loss": float(tl), "train_acc": float(ta), "val_loss": float(vl), "val_acc": float(va)}
        for (e, tl, ta, vl, va) in history_ft
    ]

meta_path.write_text(json.dumps(meta, indent=2), encoding="utf-8")

print("Saved:")
print(" -", ckpt_path)
print(" -", meta_path)

Saved:
 - outputs_02_mert_finetune_egfxset/mert_ft_last4layers_headproj.pt
 - outputs_02_mert_finetune_egfxset/metadata.json
